# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [ ]:
import os
import sys
from utils import *
sys.path.append('..')
# Set your preferred name
USER = "Lisa"
# Set the data and output directories
DATA_ROOT = '/Users/wjt/Desktop/is_data'
OUTPUT_ROOT = '/Users/wjt/Desktop/is_data/v1'
TRAIT = 'Pheochromocytoma-and-Paraganglioma'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [ ]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

# 2. Data preprocessing and selection

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [ ]:
dataset = 'GEO'
trait_subdir = "Pheochromocytoma-and-Paraganglioma"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

Repeat the below steps for all the accession numbers

In [ ]:
cohort = accession_num = "GSE39716"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

### Inital filtering and clinical data preprocessing

In [ ]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

In [ ]:
clinical_data.head()

In [ ]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [ ]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [ ]:
is_gene_availabe = True
trait_row = 2
age_row = 0
gender_row = 1

trait_type = 'binary'

In [ ]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [ ]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'tissue type: paraganglioma (PGL) tumor':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age(yrs): '):
        try:
            return float(age_string.split('age(yrs): ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: female':
        return 0
    elif gender_string == 'gender: male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age(yrs): 72'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

In [ ]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

### Genetic data preprocessing and final filtering

In [ ]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

In [ ]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [ ]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

If not required, jump directly to the gene normalization step

In [ ]:
requires_gene_mapping = True

In [ ]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [ ]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

In [ ]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'GB_LIST'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [ ]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [ ]:
from utils import *

In [ ]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

In [ ]:
ranked_df.head()

In [ ]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [ ]:
merged_data.head()

In [ ]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [ ]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

In [ ]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [ ]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)